In [1]:
import parametric as param1
import parametric2 as param2
import numpy as np
import gc

In [2]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [3]:
def is_Hermitian(m):
    """
    Checks if a given matrix is Hermitian 
    (symmetric)

    Parameters:
    -----------------------------------------
    m   :  A 2-D matrix

    Returns:
    -----------------------------------------
    logical
    """
    import numpy as np
    
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False
    
def symmetrize(m,rule="min"):
    """
    Symmetrizes a given square matrix based on a rule

    Parameters:
    -----------------------------------------
    m   :  A square matrix

    rule:  criterion for symmetrizing m
           -"min" computes the minimum of m(i,j) and m(j,i)
           -"max" computes the maximum of m(i,j) and m(j,i)
           where i and j are row and column indices

    Returns:
    -----------------------------------------
    A symmetric square matrix
    """
    import numpy as np
    
    if (m.shape[0] != m.shape[1]):
        raise ValueError("Input matrix must be square.")
        
    if (rule == "min"):
        min_mat =  np.fmin(np.triu(m),np.tril(m).T)
        return np.triu(min_mat,1) + min_mat.T

    elif (rule == "max"):
        max_mat =  np.fmax(np.triu(m),np.tril(m).T)
        return np.triu(max_mat,1) + max_mat.T
    
    else:
        raise ValueError("Specify rule as min or max.")

def fastclime_R(x,lambda_min=0.1,nlambda=50):
 
    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param1.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
   
    #Symmetrize output precision matrices 
    for i in range(maxnlambda):
        #icovlist[:,:,i] = symmetrize(iicov[:,i].reshape((d,d)).T,"min")
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    
    
def fastclime_mod(x,SigmaInput_xy,lambda_min=0.1,nlambda=50,symout = False):

    cov_input = 1
    SigmaInput_xx = x.copy()
    
    #Check if matrix is symmetric
    #if not create normalized covariance matrix
    if not is_Hermitian(SigmaInput_xx):
        SigmaInput_xx = np.corrcoef(SigmaInput_xx.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat_xx, Sigmahat_xy, mu, maxnlambda, iicov = param2.mainfunc(SigmaInput_xx,SigmaInput_xy,
                                                     lambda_min,nlambda)
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat_xx.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
   
    #Symmetrize output precision matrices 
    for i in range(maxnlambda):
        if (symout == False):
            icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
        else:
            icovlist[:,:,i] = symmetrize(iicov[:,i].reshape((d,d)).T,"min")
         
    
    gc.collect()
    
    return x, cov_input, Sigmahat_xx, Sigmahat_xy, maxnlambda, lambdamtx, icovlist
    
def fastcapme(x,y,lambda_min=0.1,nlambda=50,tau_min=0.1,ntau=50,):
    
    n = x.shape[0]
    p = y.shape[0]
    q = x.shape[1]
    
    Sxx = np.corrcoef(x.T)
    Sxy = np.cov(x.T,y.T)*(1.-1./float(n))
    
    #Estimate Gamma0
    Gammaest = fastclime_mod(Sxx,Sxy,lambda_min,nlambda,symout=False)
    
    #Estimate optimal Gamma
    Gammahat = fastclime_select(x,Gammaest[5])[1]
    
    Syy = (y-x.dot(Gammahat)).dot((y-x.dot(Gammahat)).T)/float(n)
    
    #Estimate Omega0
    Omegaest = fastclime_mod(Syy,np.eye(d),tau_min,ntau,symout=True)
    
    return Gammahat, Syy, Gammaest[4], Gammaest[5], Gammaest[6], 
           Omegaest[4], Omegaest[5], Omegaest[6]

In [35]:
np.corrcoef(np.arange(21).reshape((3,7)).T,np.arange(15).reshape((3,5)).T).shape

(12, 12)

In [4]:
L = fastclime.fastclime_generator(n = 10, d = 5)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [5]:
Pyout = fastclime1(pydat)

In [29]:
from sklearn import preprocessing
import numpy as np

In [31]:
np.cov(preprocessing.scale(pydat).T)*(1.-1./10.)

array([[ 1.        ,  0.13702914, -0.49134195, -0.49422069,  0.11948745],
       [ 0.13702914,  1.        , -0.80141566,  0.19911562,  0.21909631],
       [-0.49134195, -0.80141566,  1.        , -0.16262622, -0.18497391],
       [-0.49422069,  0.19911562, -0.16262622,  1.        , -0.44982461],
       [ 0.11948745,  0.21909631, -0.18497391, -0.44982461,  1.        ]])

In [32]:
np.cov(x.T,y.T)*(1.-1./float(n))

NameError: name 'x' is not defined

In [7]:
from numpy import matlib
#Imat = np.ndarray.flatten(np.matlib.eye(5))
Imat = np.matlib.eye(5)
Pyout2 = fastclime2(pydat,Imat)


In [8]:
Pyout2[4]

10

In [10]:
np.allclose(Pyout[4],Pyout2[5])

True

In [ ]:
Rout = fastclime.fastclime(L.rx2('data'),0.1)